## Image Embeddings

In [ ]:
import json

from os import listdir, makedirs, path

from PIL import Image as PImage, ImageOps as PImageOps

from parameters.arquigrafia import IMAGES_PATH
from parameters.embeddings import EMBEDDINGS_PATH

from models.embedding_models import Clip, EfficientNet, ResNet, Vit

makedirs(EMBEDDINGS_PATH, exist_ok=True)

### Run embeddings

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

for idx, io_file in list(enumerate(input_files))[:8192]:
  input_file_path = path.join(IMAGES_PATH, io_file)
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  image = PImageOps.exif_transpose(PImage.open(input_file_path).convert("RGB"))

  raw_embs = {}

  raw_embs["clip"] = Clip.get_embedding(image).tolist()
  raw_embs["efficient"] = EfficientNet.get_embedding(image).tolist()
  raw_embs["resnet"] = ResNet.get_embedding(image).tolist()
  raw_embs["vit"] = Vit.get_embedding(image).tolist()

  image_embs = {"raw": raw_embs}

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Reduce Dims

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

raw_embs = {}

for idx, io_file in enumerate(input_files):
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if not path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  with open(output_file_path, "r", encoding="utf8") as f:
    image_embs = json.load(f)

  for k,e in image_embs["raw"].items():
    if k not in raw_embs:
      raw_embs[k] = []
    raw_embs[k].append(e)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
clipStandard = StandardScaler()
clipPCA = PCA(n_components=4)
clipCluster = KMeans(n_clusters=6)

clip_std = clipStandard.fit_transform(raw_embs["clip"])
clip_pca = clipPCA.fit_transform(clip_std)
clip_clusters = clipCluster.fit_predict(clip_std)

# TODO: PCA
# TODO: write output

  # with open(output_file_path, "w", encoding="utf-8") as of:
  #   json.dump(image_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

In [ ]:
# 2D
for i in range(3):
  for j in range(i+1,3):
    plt.scatter(clip_pca[:,i], clip_pca[:,j], c=clip_clusters)
    plt.show()

# 3D
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

ax.scatter(clip_pca[:,0], clip_pca[:,1], clip_pca[:,2], c=clip_clusters, marker='o', linestyle='', alpha=0.5)

ax.set_title("Clip clustering")
plt.show()

## Test Embeddings

In [ ]:
imgs = [
  PImage.open(IMAGES_PATH+"/100.jpg"),
  PImage.open(IMAGES_PATH+"/101.jpg"),
  PImage.open(IMAGES_PATH+"/1000.jpg"),
  PImage.open(IMAGES_PATH+"/1001.jpg"),
  PImage.open(IMAGES_PATH+"/1010.jpg"),
  PImage.open(IMAGES_PATH+"/1011.jpg"),
]

img = imgs[0]

for i in imgs:
  display(i.resize((128,128)))

In [ ]:
emb = ResNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = ResNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = EfficientNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = EfficientNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Vit.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Vit.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Clip.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Clip.get_embedding(img)
print(emb.shape)